In [0]:
import pandas as pd
import matplotlib.pyplot as plt
import tqdm
import numpy as np
import geopy
import catboost
from math import radians, cos, sin, asin, sqrt, hypot
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import mean_absolute_error
from catboost import *
from geopy import distance
#%matplotlib notebook
from category_encoders import *
np.random.seed(0)

# **Решение**

**Сгенерированные нами признаки, которые влияют на точность:**
- hour_of_traffic "час пик" (с 8 до 10 утра и с 18 до 19 трафик более плотный) 
- direction_from_center "в центр/из центра" (направление движения в "час пик" играет большую роль - оценка завышена, если направление движения в "час пик" имеет значение 0 - "в центр")
- big_eta "предварительная оценка времени более 15 минут" (в длительных поездках отклонение реального времени от первоначальной оценки больше) 

**Признаки, которые не влияют:**
- Широта/долгота (данные зашумлены и ухудшают качество)
- Погода (не менялась практически на протяжении всего датасета, поэтому вычеркиваем)
- Расстояние до центра (ухудшает качество, добавляет шум, убираем)
- День недели, дата (не прибавляют к качеству)


In [0]:
#Функции для расчета сгенерированных нами признаков 
# - расстояние до центра (отдельно для точки отправления и для точки назначения)
# - направление движения (в центр/из центра)
# - час пик или нет
# - предварительная оценка больше 15 минут или нет

def to_center_dist(lat1, lat2, lon1, lon2):
        dist = 0
        dist += geodesic(lat1, lon1, ellipsoid='WGS-84').m
        dist += geodesic(lat2, lon2, ellipsoid='WGS-84').m
        return dist

def distancer(row):
    coords_1 = (row['latitude'], row['longitude'])
    coords_2 = (row['center_latitude'], row['center_longitude'])
    return geopy.distance.VincentyDistance(coords_1, coords_2).miles

def del_distancer(row):
    coords_1 = (row['del_latitude'], row['del_longitude'])
    coords_2 = (row['center_latitude'], row['center_longitude'])
    return geopy.distance.VincentyDistance(coords_1, coords_2).miles   

def direction(row):
    direction = row['del_distance'] - row['distance']
    if direction > 0:  
        return 1
    if direction < 0:
        return 0

def hour_of_traffic(row):
    hour = row['hour']
    if hour == 9 or hour == 18:  
        return 1
    else:
        return 0     

def big_eta(row):
    hour = row['ETA']
    if hour >= 1000:  
        return 1
    else:
        return 0             

      

In [107]:
# Время исполнения ячейки ~ 10-15 минут.
# Подготавливаем датасет для обучения и добавляем признаки

class DataLoader():
    def __init__(self, time_order=None):
        '''
        time_order: добавляет признак, в какое время суток был сделан заказ такси.
        '''
        self.time_order = time_order
        self.train, self.test = self.prepare_train(pd.read_csv('/content/drive/My Drive/train.csv', index_col=0), time_order), self.prepare_test(pd.read_csv('/content/drive/My Drive/test_additional.csv', index_col=0))
        self.bad_points = None
        
    

    def prepare_train(self, df, dist_to_bad=None):
        df['hour'] = pd.DatetimeIndex(pd.to_datetime(df['OrderedDate'])).hour
        df['day'] = pd.DatetimeIndex(pd.to_datetime(df['OrderedDate'])).day
        df['weekday'] = pd.DatetimeIndex(pd.to_datetime(df['OrderedDate'])).weekday
        df['distance'] = df.apply(distancer, axis=1)  
        df['del_distance'] = df.apply(del_distancer, axis=1) 
        df['direction_from_center'] = df.apply(direction, axis=1) 
        df['hour_of_traffic'] = df.apply(hour_of_traffic, axis=1)
        df['big_eta'] = df.apply(big_eta, axis=1)

        df = df.drop(['OrderedDate', 'center_latitude', 'center_longitude', 'latitude', 'longitude', 'del_latitude', 'del_longitude'], axis=1)
        if dist_to_bad is None:
            return df
        print('Подготовка трейн датасета')
        self.time_order = 'trained'
        return df
    
    def prepare_test(self, df):
        df['hour'] = pd.DatetimeIndex(pd.to_datetime(df['OrderedDate'])).hour
        df['day'] = pd.DatetimeIndex(pd.to_datetime(df['OrderedDate'])).day
        df['weekday'] = pd.DatetimeIndex(pd.to_datetime(df['OrderedDate'])).weekday
        df['distance'] = df.apply(distancer, axis=1)
        df['del_distance'] = df.apply(del_distancer, axis=1) 
        df['direction_from_center'] = df.apply(direction, axis=1) 
        df['hour_of_traffic'] = df.apply(hour_of_traffic, axis=1)
        df['big_eta'] = df.apply(big_eta, axis=1)
        df = df.drop(['OrderedDate', 'center_latitude', 'center_longitude', 'latitude', 'longitude', 'del_latitude', 'del_longitude'], axis=1)
        return df
    
data = DataLoader(1200)

Подготовка трейн датасета


In [0]:
train_data = data.train
test_data = data.test

In [0]:
from category_encoders import *

In [0]:
# Класс Coder помогает перебирать различные способы кодировки, чтобы тестировать влияние кодировки на целевую переменную.
class Coder():
    def __init__(self, train_, test, labels, cols):
        self.train, self.test, self.labels, self.cols = train, test, labels, cols
        
    def base(self):
        cod = BaseNEncoder(cols=self.cols).fit(self.train, self.labels)
        return cod.transform(self.train), cod.transform(self.test)
    
    def target_enc(self):
        cod = TargetEncoder(cols=self.cols).fit(self.train, self.labels)
        return cod.transform(self.train), cod.transform(self.test)
    
    def backward(self):
        cod = BackwardDifferenceEncoder(cols=self.cols).fit(self.train, self.labels)
        return cod.transform(self.train), cod.transform(self.test)
    
    def james_stein(self):
        cod = JamesSteinEncoder(cols=self.cols).fit(self.train, self.labels)
        return cod.transform(self.train), cod.transform(self.test)
    
    def m_estimate(self):
        cod = MEstimateEncoder(cols=self.cols).fit(self.train, self.labels)
        return cod.transform(self.train), cod.transform(self.test)

In [0]:
train =  train_data[['main_id_locality', 'ETA', 'EDA', 'hour', 'direction_from_center', 'big_eta', 'hour_of_traffic']]
label = train_data['RTA']
X_train, X_test, y_train, y_test = train_test_split(train, label)

In [110]:
# Обучение 
reg = catboost.CatBoostRegressor(iterations=5000, learning_rate=0.001, depth=10, loss_function='MAPE')
pool = catboost.Pool(X_test.values, y_test.values)
reg.fit(X_train, y_train, eval_set=pool)

0:	learn: 0.4323172	test: 0.4327543	best: 0.4327543 (0)	total: 322ms	remaining: 15.8s
1:	learn: 0.4321223	test: 0.4325597	best: 0.4325597 (1)	total: 620ms	remaining: 14.9s
2:	learn: 0.4320747	test: 0.4325126	best: 0.4325126 (2)	total: 932ms	remaining: 14.6s
3:	learn: 0.4319722	test: 0.4324109	best: 0.4324109 (3)	total: 1.23s	remaining: 14.1s
4:	learn: 0.4317559	test: 0.4321948	best: 0.4321948 (4)	total: 1.53s	remaining: 13.8s
5:	learn: 0.4317069	test: 0.4321462	best: 0.4321462 (5)	total: 1.82s	remaining: 13.4s
6:	learn: 0.4315696	test: 0.4320095	best: 0.4320095 (6)	total: 2.12s	remaining: 13s
7:	learn: 0.4314606	test: 0.4319015	best: 0.4319015 (7)	total: 2.42s	remaining: 12.7s
8:	learn: 0.4312723	test: 0.4317134	best: 0.4317134 (8)	total: 2.74s	remaining: 12.5s
9:	learn: 0.4311244	test: 0.4315656	best: 0.4315656 (9)	total: 3.04s	remaining: 12.2s
10:	learn: 0.4309936	test: 0.4314347	best: 0.4314347 (10)	total: 3.36s	remaining: 11.9s
11:	learn: 0.4308585	test: 0.4313003	best: 0.4313003 (

In [0]:
df_test= test_data[['main_id_locality', 'ETA', 'EDA', 'hour', 'direction_from_center', 'big_eta', 'hour_of_traffic']]
vals = reg.predict(df_test.values)

In [0]:
import numpy as np

def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

mean_absolute_percentage_error(df_validation['RTA'].values[:89938], vals)

In [0]:
# submission
answers = pd.DataFrame(vals, columns=['Prediction'])
answers.to_csv('sub.csv', index_label='Id')